In [1]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
split test into validation

In [2]:
train_df = pd.read_csv('train-balanced-sarcasm.csv')

In [3]:
train_df.head(10)

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...
5,0,"I don't pay attention to her, but as long as s...",only7inches,AskReddit,0,0,0,2016-09,2016-09-02 10:35:08,do you find ariana grande sexy ?
6,0,Trick or treating in general is just weird...,only7inches,AskReddit,1,-1,-1,2016-10,2016-10-23 21:43:03,What's your weird or unsettling Trick or Treat...
7,0,Blade Mastery+Masamune or GTFO!,P0k3rm4s7,FFBraveExvius,2,-1,-1,2016-10,2016-10-13 21:13:55,Probably Sephiroth. I refuse to taint his grea...
8,0,"You don't have to, you have a good build, buy ...",SoupToPots,pcmasterrace,1,-1,-1,2016-10,2016-10-27 19:11:06,What to upgrade? I have $500 to spend (mainly ...
9,0,I would love to see him at lolla.,chihawks,Lollapalooza,2,-1,-1,2016-11,2016-11-21 23:39:12,Probably count Kanye out Since the rest of his...


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010826 entries, 0 to 1010825
Data columns (total 10 columns):
label             1010826 non-null int64
comment           1010773 non-null object
author            1010826 non-null object
subreddit         1010826 non-null object
score             1010826 non-null int64
ups               1010826 non-null int64
downs             1010826 non-null int64
date              1010826 non-null object
created_utc       1010826 non-null object
parent_comment    1010826 non-null object
dtypes: int64(4), object(6)
memory usage: 77.1+ MB


In [8]:
train_df.dropna(subset=['comment'], inplace=True)

In [9]:
train_df['label'].value_counts()

0    505405
1    505368
Name: label, dtype: int64

In [81]:
clean_df = train_df.drop(['author', 'subreddit', 'ups', 'downs','date','created_utc', 'parent_comment','label'], axis=1)
print(clean_df)

                                                   comment  score
0                                               NC and NH.      2
1        You do know west teams play against west teams...     -4
2        They were underdogs earlier today, but since G...      3
3        This meme isn't funny none of the "new york ni...     -8
4                          I could use one of those tools.      6
5        I don't pay attention to her, but as long as s...      0
6            Trick or treating in general is just weird...      1
7                          Blade Mastery+Masamune or GTFO!      2
8        You don't have to, you have a good build, buy ...      1
9                        I would love to see him at lolla.      2
10       I think a significant amount would be against ...     92
11                          Damn I was hoping God was real     14
12                                    They have an agenda.      4
13                                             Great idea!      1
14       A

In [82]:
_, data_X, _, data_y = \
        train_test_split(clean_df, train_df['label'], test_size=0.01, random_state=24)

In [84]:
print (data_X.shape)


(10108, 2)


In [85]:
train_X, test_X, train_y, test_y = train_test_split(data_X, data_y, test_size=0.2, random_state=24)

In [96]:
print(train_X.shape)
print(train_X.values)
print(train_X.values[:,0])

(8086, 2)
[["Yeah, that's too many." 1]
 ['In Isreali eyes there is no such thing as a *credible* palestinian...' 2]
 [ 'Nah the fact that Freddie Gray was a know crack dealer had nothing to do with it, clearly they just picked out some random black dude!'
  3]
 ..., 
 ['The sounds still haunts me.' 2]
 ['Shame on you for asking an insightful question...' 4]
 ["Because It doesn't have a lot of support?" 0]]
["Yeah, that's too many."
 'In Isreali eyes there is no such thing as a *credible* palestinian...'
 'Nah the fact that Freddie Gray was a know crack dealer had nothing to do with it, clearly they just picked out some random black dude!'
 ..., 'The sounds still haunts me.'
 'Shame on you for asking an insightful question...'
 "Because It doesn't have a lot of support?"]


In [97]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer() 
X_train = vectorizer.fit_transform(train_X.values[:,0])

CPU times: user 218 ms, sys: 27.7 ms, total: 245 ms
Wall time: 262 ms


In [100]:
print(X_train.shape)

(8086, 12060)


In [63]:
y_train = np.array(train_y)

In [64]:
y_train.shape

(8086,)

In [70]:
%%time
from sklearn.decomposition import TruncatedSVD # PCA does not support sparse input
n_components = 1000
svd = TruncatedSVD(n_components=n_components).fit(X_train)

CPU times: user 17.1 s, sys: 756 ms, total: 17.9 s
Wall time: 11.3 s


In [71]:
svd.explained_variance_ratio_.sum() 

0.59166615093439856

In [128]:
print(X_train.shape)
print(train_X.values[:,1].shape)

(8086, 12060)
(8086,)


In [127]:
from scipy.sparse import hstack
features = hstack((X_train ,train_X.values[:,1]))

ValueError: blocks[0,:] has incompatible row dimensions. Got blocks[0,1].shape[0] == 1, expected 8086.

In [125]:
print(X_train[0])

  (0, 11983)	0.44750141809
  (0, 10710)	0.313037719578
  (0, 10896)	0.542759365913
  (0, 6552)	0.638092577642


In [126]:
print(features)

[ <8086x12060 sparse matrix of type '<class 'numpy.float64'>'
	with 75528 stored elements in Compressed Sparse Row format>
 1 2 ..., 2 4 0]


In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()